In [ ]:
#https://engkimbs.tistory.com/625

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import pickle
import time
import FinanceDataReader as fdr
import io

In [3]:
import seaborn as sns

In [4]:
def se(df,start,end):
    df = df[(df.index>=start)*(df.index<=end)].reset_index()
    
    return(df)

### 재무제표 크롤링

In [63]:
url = "https://finance.naver.com/item/coinfo.nhn?code=005930&target=finsum_more"
url = "https://finance.naver.com/item/coinfo.nhn?code=005930"
url = "https://finance.naver.com/item/main.nhn?code=005930"

In [65]:
soup = bs(requests.get(url).text,'lxml')

In [86]:
table = soup.find("div",{"class":"section cop_analysis"})
table = table.find('div',{"class":"sub_section"})

In [218]:
jaemu = []

In [219]:
dump=[]

In [ ]:
for idx,code in enumerate(codes):

    URL = "https://finance.naver.com/item/main.nhn?code="+code

    samsung_electronic = requests.get(URL)
    html = samsung_electronic.text

    soup = BeautifulSoup(html, 'lxml')
    try:
        finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]
    
    except IndexError:
        print('error : ',code)
        dump.append(code)
        time.sleep(10)
        continue


    th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
    annual_date = th_data[3:7]
    quarter_date = th_data[7:13]




    finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]


    finance_data = [item.get_text().strip() for item in finance_html.select('td')]

    finance_data = np.array(finance_data)
    finance_data.resize(len(finance_index), 10)
    finance_date = annual_date + quarter_date
    
    try:
        finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index, columns=finance_date)
    except:
        print('error2 : ',code)
        dump.append(code)
        continue

    jaemu.append({'code':code,'df':finance})
    
    if(idx%10==9):
        print(idx/len(codes))
        time.sleep(5)
    #time.sleep(np.random.rand(1)[0]*0.8)


In [254]:
with open('jaemu_0511.pkl','wb') as f:
    pickle.dump(jaemu,f)

In [255]:
with open('dump.pkl_0511','wb') as f:
    pickle.dump(dump,f)

In [ ]:
codes_ = list(x for x in codes if x not in dump)

In [ ]:
#KB손해보험 주식 1주당 0.57287주의 KB금융지주 주식으로 교환할 수 있게 됨
#지주회사의 완전자사화 등 -> 상폐x

### 일일 주가 데이터 - 코스피

In [5]:
'''
code
kospi : STK
kosdaq : KSQ
'''
def closedJongmok(code):
    url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
                'name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&' \
                'market_gubun='+code+'&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&' \
                'isu_srt_cd=&fromdate=20000101&todate=22001231&del_cd=1&' \
                'pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

    header_data = {
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.get(url, headers=header_data)

    # STEP 02: download
    url = 'http://file.krx.co.kr/download.jspx'
    form_data = {'code': r.text}
    header_data = {
        'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.post(url, data=form_data, headers=header_data)
    df = pd.read_excel(io.BytesIO(r.content))
    df['종목코드'] = df['종목코드'].str.replace('A', '')
    df['폐지일'] = pd.to_datetime(df['폐지일'])
    col_map = {'종목코드':'Symbol', '기업명':'Name', '폐지일':'DelistingDate', '폐지사유':'Reason'}
    df=df.rename(columns = col_map)
    return(df)

In [6]:
def closedJongmok(code):
    url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
                'name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&' \
                'market_gubun='+code+'&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&' \
                'isu_srt_cd=&fromdate=20000101&todate=22001231&del_cd=1&' \
                'pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

    header_data = {
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.get(url, headers=header_data)
    
    url = 'http://file.krx.co.kr/download.jspx'
    form_data = {'code': r.text}
    header_data = {
        'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.post(url, data=form_data, headers=header_data)
    df = pd.read_excel(io.BytesIO(r.content))
    df['종목코드'] = df['종목코드'].str.replace('A', '')
    df['폐지일'] = pd.to_datetime(df['폐지일'])
    col_map = {'종목코드':'Symbol', '기업명':'Name', '폐지일':'DelistingDate', '폐지사유':'Reason'}
    df=df.rename(columns = col_map)
    return df

In [79]:
code= '079160'

In [100]:
temp = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=csv&url=MKD/13/1302/13020303/mkd13020303&isu_cdnm=A003550%2FLG&isu_cd=KR7003550001&isu_nm=LG&isu_srt_cd=A003550&period_selector=day&fromdate=20200801&todate=20200901&pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020303%2FMKD13020303.jsp'


In [101]:
temp

'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=csv&url=MKD/13/1302/13020303/mkd13020303&isu_cdnm=A003550%2FLG&isu_cd=KR7003550001&isu_nm=LG&isu_srt_cd=A003550&period_selector=day&fromdate=20200801&todate=20200901&pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020303%2FMKD13020303.jsp'

In [107]:
url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
'name=fileDown&filetype=xls&url=MKD/13/1302/13020303/mkd13020303&' \
'isu_cdnm=%2F%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&' \
'isu_cd=KR7003550002&isu_nm=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&' \
'isu_srt_cd=&period_selector=day&fromdate=20200801&todate=20200901&' \
'pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020303%2FMKD13020303.jsp'


'079160'

In [112]:
len('KR 7 00355 000 2')

12

In [108]:
url

'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=xls&url=MKD/13/1302/13020303/mkd13020303&isu_cdnm=%2F%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&isu_cd=KR7003550002&isu_nm=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&isu_srt_cd=&period_selector=day&fromdate=20200801&todate=20200901&pagePath=%2Fcontents%2FMKD%2F13%2F1302%2F13020303%2FMKD13020303.jsp'

In [109]:
header_data = {
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}
r = requests.get(url, headers=header_data)

In [110]:
url = 'http://file.krx.co.kr/download.jspx'
form_data = {'code': r.text}
header_data = {
    'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}
r = requests.post(url, data=form_data, headers=header_data)
df = pd.read_excel(io.BytesIO(r.content))
df=df.rename(columns = col_map)


In [ ]:
def tradeWho(code):
    url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
                'name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&' \
                'market_gubun='+code+'&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&' \
                'isu_srt_cd=&fromdate=20000101&todate=22001231&del_cd=1&' \
                'pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

    header_data = {
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.get(url, headers=header_data)
    
    url = 'http://file.krx.co.kr/download.jspx'
    form_data = {'code': r.text}
    header_data = {
        'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
        'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
    }
    r = requests.post(url, data=form_data, headers=header_data)
    df = pd.read_excel(io.BytesIO(r.content))
    df['종목코드'] = df['종목코드'].str.replace('A', '')
    df['폐지일'] = pd.to_datetime(df['폐지일'])
    col_map = {'종목코드':'Symbol', '기업명':'Name', '폐지일':'DelistingDate', '폐지사유':'Reason'}
    df=df.rename(columns = col_map)
    return df

In [35]:
code = '225650'

In [36]:
url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
            'name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&' \
            'market_gubun='+code+'&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&' \
            'isu_srt_cd=&fromdate=20000101&todate=22001231&del_cd=1&' \
            'pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

header_data = {
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}
r = requests.get(url, headers=header_data)

In [40]:
temp = 'https://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=csv&url=MKD/04/0406/04060600/mkd04060600&market_gubun=ALL&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&isu_srt_cd=&fromdate=20190902&todate=20200902&del_cd=1&pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.js'


In [37]:
url = 'http://file.krx.co.kr/download.jspx'
form_data = {'code': r.text}
header_data = {
    'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}
r = requests.post(url, data=form_data, headers=header_data)
df = pd.read_excel(io.BytesIO(r.content))
df['종목코드'] = df['종목코드'].str.replace('A', '')
df['폐지일'] = pd.to_datetime(df['폐지일'])
col_map = {'종목코드':'Symbol', '기업명':'Name', '폐지일':'DelistingDate', '폐지사유':'Reason'}
df=df.rename(columns = col_map)

In [13]:
import requests
import pandas as pd
import io

In [14]:
closedJongmok('225650')

,Symbol,Name,DelistingDate,Reason
0,277480,신한제4호스팩,2020-08-20,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소
1,204990,코썬바이오,2020-08-11,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정
2,228180,티씨엠생명과학,2020-08-07,포괄적 주식교환에 따른 상장폐지 신청('20.7.17)
3,065940,바이오빌,2020-07-31,감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)
4,101360,이엔드디,2020-07-30,코스닥시장 이전상장
...,...,...,...,...
1267,034260,삼성다이나믹,2000-01-11,증권투자회사법 64조에 의한 해산사유 발생
1268,034320,에머랄드 1호,2000-01-11,증권투자회사법 64조에 의한 해산사유 발생
1269,034270,삼성 프라임,2000-01-11,증권투자회사법 64조에 의한 해산사유 발생
1270,014590,나자인,2000-01-06,증권거래소 상장


In [118]:
kospiDf = fdr.StockListing('KOSPI')
kospiDf = kospiDf[kospiDf.apply(lambda x:len(x['Symbol'])==6,axis=1)] #이상한 몇호 뭐시기 이런거 들어있어서 제외
codes = kospiDf['Symbol'].tolist()

In [121]:
codes

['095570',
 '006840',
 '027410',
 '282330',
 '138930',
 '001460',
 '001465',
 '001040',
 '079160',
 '00104K',
 '000120',
 '011150',
 '011155',
 '001045',
 '097950',
 '097955',
 '000590',
 '012030',
 '016610',
 '005830',
 '000990',
 '000995',
 '139130',
 '001530',
 '004840',
 '155660',
 '069730',
 '017940',
 '007700',
 '114090',
 '078930',
 '006360',
 '001250',
 '007070',
 '078935',
 '012630',
 '039570',
 '089470',
 '294870',
 '011200',
 '082740',
 '003560',
 '175330',
 '234080',
 '001060',
 '001067',
 '001065',
 '096760',
 '105560',
 '002380',
 '009070',
 '009440',
 '119650',
 '092220',
 '016380',
 '016385',
 '001390',
 '001940',
 '025000',
 '092230',
 '000040',
 '044450',
 '030200',
 '033780',
 '030210',
 '058850',
 '058860',
 '093050',
 '003550',
 '034220',
 '001120',
 '051900',
 '051905',
 '003555',
 '032640',
 '011070',
 '066570',
 '066575',
 '108670',
 '108675',
 '037560',
 '051910',
 '051915',
 '079550',
 '006260',
 '010120',
 '000680',
 '229640',
 '023150',
 '035420',
 '181710',

In [7]:
#상폐 데이터
closedDf = closedJongmok('STK')
closedDf = closedDf[closedDf.apply(lambda x:len(x['Symbol'])==6,axis=1)]

codesClosed = closedDf['Symbol'].tolist()

In [8]:
len(codes+codesClosed)

1304

In [28]:
price=[]
for idx,code in enumerate(codes):
    element = {"code":code, "price":fdr.DataReader(code, '2000')}
    price.append(element)
    if(idx%10==9):
        print(idx,end='$')

9$19$29$39$49$59$69$79$89$99$109$119$129$139$149$159$169$179$189$199$209$219$229$239$249$259$269$279$289$299$309$319$329$339$349$359$369$379$389$399$409$419$429$439$449$459$469$479$489$499$509$519$529$539$549$559$569$579$589$599$609$619$629$639$649$659$669$679$689$699$709$719$729$739$749$759$769$779$789$799$809$819$829$839$849$859$869$879$889$899$909$

In [ ]:
#2000년 이후 , 현재 상장
price = []
for idx,code in enumerate(codes+codesClosed):
    element = {"code":code, "price":fdr.DataReader(code, '2000')}
    price.append(element)
    if(idx%10==9):
        print(idx,end='$')

In [29]:
with open('/home/edward/stock/data/price_kos_0702.pkl','wb') as f:
     pickle.dump(price,f)

In [8]:
with open("price_kos_0823.pkl","rb") as f:
    price = pickle.load(f)

In [31]:
#종가를 기준으로한 data frame
check=0

timePriceDf = pd.DataFrame()

for i in price:
    if(len(i['price']) >0):
        temp = pd.DataFrame(i['price']['Close'])
        temp.columns = [i['code']]
        timePriceDf = pd.concat([timePriceDf,temp],axis=1)
    
    check+=1
    if(check%100==1):
        print(check, end='$')
    

1$101$201$301$401$501$601$701$801$901$

In [32]:
timePriceDf.to_csv("/home/edward/stock/data/kospi_close_0823.csv")

In [33]:
#종가를 기준으로한 data frame
check=0

timePriceDf = pd.DataFrame()

for i in price:
    if(len(i['price']) >0):
        temp = pd.DataFrame(i['price']['Open'])
        temp.columns = [i['code']]
        timePriceDf = pd.concat([timePriceDf,temp],axis=1)
    
    check+=1
    if(check%100==1):
        print(check, end='$')
    

1$101$201$301$401$501$601$701$801$901$

In [34]:
timePriceDf.to_csv("/home/edward/stock/data/kospi_open_0823.csv")

In [35]:
#종가를 기준으로한 data frame
check=0

timePriceDf = pd.DataFrame()

for i in price:
    if(len(i['price']) >0):
        temp = pd.DataFrame(i['price']['Volume'])
        temp.columns = [i['code']]
        timePriceDf = pd.concat([timePriceDf,temp],axis=1)
    
    check+=1
    if(check%100==1):
        print(check, end='$')
    

1$101$201$301$401$501$601$701$801$901$

In [36]:
timePriceDf.to_csv("/home/edward/stock/data/kospi_volume_0823.csv")

In [10]:
#종가를 기준으로한 data frame
check=0

timePriceDf = pd.DataFrame()

for i in price:
    if(len(i['price']) >0):
        temp = pd.DataFrame(i['price']['High'])
        temp.columns = [i['code']]
        timePriceDf = pd.concat([timePriceDf,temp],axis=1)
    
    check+=1
    if(check%100==1):
        print(check, end='$')
    

1$101$201$301$401$501$601$701$801$901$

In [11]:
timePriceDf.to_csv("/home/edward/stock/data/kospi_high_0823.csv")

In [37]:
df_krx = fdr.StockListing('KRX')

In [49]:
kospiCodes = df_krx[df_krx['Market']=='KOSPI'][['Symbol','Name','Industry']]

In [51]:
kospiCodes.to_csv("/home/edward/stock/data/kospi_codes.csv",index=False)

### 일일 주가 데이터 - 코스닥

In [16]:
kosdaqDf = fdr.StockListing('KOSDAQ')

codes = kosdaqDf['Symbol'].tolist()

In [19]:
#상폐 데이터
closedDf = closedJongmok('KSQ')
closedDf = closedDf[closedDf.apply(lambda x:len(x['Symbol'])==6,axis=1)]
codesClosed = closedDf['Symbol'].tolist()

In [20]:
#2010년 이후 , 현재 상장
price = []
for idx,code in enumerate(codes+codesClosed):
    element = {"code":code, "price":fdr.DataReader(code, '2010')}
    price.append(element)
    if(idx%10==9):
        print(idx,end='$')

9$19$29$39$49$59$69$79$89$99$109$119$129$139$149$159$169$179$189$199$209$219$229$239$249$259$269$279$289$299$309$319$329$339$349$359$369$379$389$399$409$419$429$439$449$459$469$479$489$499$509$519$529$539$549$559$569$579$589$599$609$619$629$639$649$659$669$679$689$699$709$719$729$739$749$759$769$779$789$799$809$819$829$839$849$859$869$879$889$899$909$919$929$939$949$959$969$979$989$999$1009$1019$1029$1039$1049$1059$1069$1079$1089$1099$1109$1119$1129$1139$1149$1159$1169$1179$1189$1199$1209$1219$1229$1239$1249$1259$1269$1279$1289$1299$1309$1319$1329$1339$1349$1359$1369$1379$1389$1399$1409$1419$1429$1439$1449$1459$1469$1479$1489$1499$1509$1519$1529$1539$1549$1559$1569$1579$1589$1599$1609$1619$1629$1639$1649$1659$1669$1679$1689$1699$1709$1719$1729$1739$1749$1759$1769$1779$1789$1799$1809$1819$1829$1839$1849$1859$1869$1879$1889$1899$1909$1919$1929$1939$1949$1959$1969$1979$1989$1999$2009$2019$2029$2039$2049$2059$2069$2079$2089$2099$2109$2119$2129$2139$2149$2159$2169$2179$

In [22]:
with open('/Users/edward/large_data/price_daq_0702.pkl','wb') as f:
    pickle.dump(price,f)

In [4]:
# temp = fdr.DataReader('008080','2000')

# temp[temp.index >='2010'] #이상한데이터

In [156]:
#종가를 기준으로한 data frame
check=0

timePriceDf = pd.DataFrame()

for i in price:
    if(len(i['price']) >0):
        temp = pd.DataFrame(i['price']['Close'])
        temp.columns = [i['code']]
        timePriceDf = pd.concat([timePriceDf,temp],axis=1)
    
    check+=1
    if(check%100==1):
        print(check, end='$')
    

1$101$201$301$401$501$601$701$801$901$1001$1101$1201$1301$1401$1501$1601$1701$1801$1901$2001$2101$

In [158]:
timePriceDf

,060310,054620,265520,211270,035760,051500,058820,023460,065770,083660,...,034300,017370,024720,033180,021240,010040,013310,003380,014590,016450
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,NaN,NaN,NaN,NaN,127817.0,NaN,NaN,2686.0,NaN,NaN,...,6100.0,NaN,2434.0,7928.0,2890.0,1503.0,NaN,NaN,25000.0,42000.0
2000-01-05,NaN,NaN,NaN,NaN,113848.0,NaN,NaN,2644.0,NaN,NaN,...,5600.0,NaN,2296.0,7183.0,2700.0,1383.0,NaN,NaN,26500.0,41500.0
2000-01-06,NaN,NaN,NaN,NaN,109133.0,NaN,NaN,2476.0,NaN,NaN,...,5390.0,NaN,2388.0,6461.0,2590.0,1545.0,NaN,NaN,1736.0,1549.0
2000-01-07,NaN,NaN,NaN,NaN,96910.0,NaN,NaN,2434.0,NaN,NaN,...,5210.0,NaN,2572.0,5708.0,2320.0,1600.0,NaN,NaN,1479.0,1318.0
2000-01-10,NaN,NaN,NaN,NaN,99006.0,NaN,NaN,2392.0,NaN,NaN,...,5250.0,NaN,2526.0,5867.0,2280.0,1586.0,NaN,NaN,1261.0,1122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-22,2800.0,8490.0,26450.0,7310.0,118000.0,20100.0,4170.0,1545.0,2510.0,4170.0,...,22950.0,3260.0,22200.0,4780.0,64200.0,3470.0,2250.0,6900.0,NaN,5090.0
2020-05-25,2775.0,8460.0,26500.0,7450.0,118300.0,20050.0,4230.0,1560.0,2475.0,4170.0,...,23000.0,3395.0,22100.0,4815.0,63400.0,3660.0,2305.0,7540.0,NaN,5170.0
2020-05-26,2775.0,8470.0,26750.0,7630.0,118500.0,20100.0,4220.0,1580.0,2470.0,4170.0,...,23450.0,3500.0,24000.0,5000.0,70500.0,3700.0,2315.0,7640.0,NaN,5230.0


In [157]:
timePriceDf.to_csv("./data/kosdaq_2000_0528.csv")

### 업종별 per

industry별로 list 만들어서 유사도 검정 or 몇개 테마만 골라

In [6]:
kospiDf = pd.read_csv("./data/upjong_0623.csv")

In [85]:
kospiDf=kospiDf.dropna()

In [87]:
kospiDf[kospiDf['Name']=='셀트리온헬스케어']

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1104,091990,KOSDAQ,셀트리온헬스케어,기타 전문 도매업,바이오의약품 마케팅 및 판매,2017-07-28,12월,김형기,http://www.celltrionhealthcare.com/kr/index.do,인천광역시


In [92]:
kospiDf[kospiDf.apply(lambda x:'자동차' in x['Industry'],axis=1)]

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
30,005830,KOSPI,DB손해보험,보험업,"자동차보험,화재보험,해상보험,특종보험,장기보험",1973-06-28,12월,김정남,http://www.idbins.com,서울특별시
48,130500,KOSDAQ,GH신소재,기타 섬유제품 제조업,"자동차용 부직포, 폴리우레탄 폼",2013-08-02,12월,우희구,http://www.gumho-nt.com,경상북도
62,089470,KOSPI,HDC현대EP,기타 화학제품 제조업,"합성수지(PP컴파운딩가공,자동차범퍼,내장제) 제조,도소매",2006-09-25,12월,김 명 호(직무대행 선임),http://www.hyundai-ep.com,충청남도
64,011200,KOSPI,HMM,해상 운송업,"정기,부정기,전용선,콘테이너선,자동차선,LNG선 해운",1995-10-05,12월,배재훈,http://www.hmm21.com,서울특별시
97,009070,KOSPI,KCTC,도로 화물 운송업,"화물자동차운송,항만하역,창고보관,컨테이너조작,중량물해상운송",1978-09-29,12월,이준환,http://www.kctc.co.kr,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2492,001450,KOSPI,현대해상,보험업,손해보험(자동차보험),1989-08-25,12월,"조용일, 이성재",http://www.hi.co.kr,서울특별시
2506,013520,KOSPI,화승알앤에이,자동차 신품 부품 제조업,"자동차용 고무제품(제조저압호스,고압호스,산업용고무호스,,WEATHER STIRP,고...",1991-02-22,12월,"현지호,이정두",http://www.hsrna.com,경상남도
2509,010690,KOSPI,화신,자동차 신품 부품 제조업,"자동차엔진부품(ARM류,C/MBR류),자동차부품 제조",1994-01-14,12월,"정서진, 장의호",http://www.hwashin.co.kr,경상북도
2510,126640,KOSDAQ,화신정공,자동차 신품 부품 제조업,자동차부품,2010-08-31,12월,정 서 진,http://hsp.hwashin.co.kr,경상북도


In [11]:
kospi = kospiDf[kospiDf['Market']=='KOSPI']

In [79]:
soft = kospiDf[kospiDf['Sector']=='특수 목적용 기계 제조업']
soft=soft.dropna()

In [22]:
jejo = kospiDf.dropna()[kospiDf.dropna().apply(lambda x:'자동차'  in x['Sector'], axis=1)]

In [24]:
kospiDf.groupby('Sector').size().reset_index().sort_values(0,ascending = False).head(30)


,Sector,0
141,특수 목적용 기계 제조업,137
78,소프트웨어 개발 및 공급업,129
125,전자부품 제조업,120
111,자동차 신품 부품 제조업,97
104,의약품 제조업,91
34,기타 금융업,88
47,기타 화학제품 제조업,86
25,금융 지원 서비스업,77
140,통신 및 방송 장비 제조업,73
57,반도체 제조업,57


In [178]:
code = ([x for x in jejo if x in codeTemp])

In [179]:
df = perTimeDf[code].mean().reset_index()

df.columns = ['code','per']

In [180]:
reluMean(df['per'])

/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


(27.216100100284958, 0.15753424657534246)

In [114]:
def upjong(name):

    codeTemp = perTimeDf.columns.tolist()
    code = kospiDf[kospiDf['Sector'] == name]['Symbol'].tolist()

    code = ([x for x in code if x in codeTemp])

    df = perTimeDf[code].mean().reset_index()

    df.columns = ['code','per']
    return(df)

In [112]:
def reluMean(arr):
    arr=np.array(arr)
    return (np.mean(arr[arr>0]), len(arr[arr<0])/len(arr))

In [152]:
reluMean(upjong('기타 식품 제조업')['per'].tolist())

(36.94565108362261, 0.0)

In [130]:
temp = perTimeDf['272210'][:5].values

In [148]:
upjong('금융 지원 서비스업')

,code,per
0,016610,41.169215
1,030210,8.765298
2,005940,12.188581
3,001510,18.105961
4,030610,4.814256
5,003540,9.720416
6,008560,7.515140
7,006800,11.514433
8,001270,8.678502
9,016360,11.128393


In [121]:
reluMean(upjong('전자부품 제조업')['per'].tolist())

/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/edward/opt/anaconda3/envs/edward_park/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


(27.68780637126737, 0.21052631578947367)

In [99]:
arr = medicalDf['per'].tolist()

### 2018년 per 

In [42]:
timePriceDf = pd.read_csv("kospi18_0519.csv", index_col = 0)

In [52]:
#per, pbr, eps 등 지수와 코드 가져오는 함수
def indexYear(index, year):
    codes = []
    lists = []

    for i in range(len(jaemu)):
        
        try: 
            jipyo = jaemu[i]['df'].loc[index, year]

            if(type(jipyo) == str):
                if(jipyo==''):
                    continue

                lists.append(jipyo)
                codes.append(jaemu[i]['code'])

            elif(jipyo.tolist()[0]!=''):
                lists.append(jipyo.tolist()[0])
                codes.append(jaemu[i]['code'])



        except KeyError:
            continue
    return(codes, lists)

In [45]:
with open('jaemu_0511.pkl','rb') as f:
    jaemu = pickle.load(f)

In [53]:
indexDf = pd.DataFrame({'code':indexYear('EPS(원)','2017.12')[0],
                        'index_2017':indexYear('EPS(원)','2017.12')[1]})

In [55]:
indexDf['index_2017'] = indexDf['index_2017'].str.replace(',','')
indexDf['index_2017'] = indexDf['index_2017'].astype(float)
indexDf = indexDf[indexDf['index_2017'] !=0]

In [57]:
perTimeDf = timePriceDf[indexDf['code']]/indexDf['index_2017'].tolist() #per 10기준으로 판단

In [61]:
perTimeDf = perTimeDf[perTimeDf.index<'2019']

In [62]:
perTimeDf

,095570,006840,027410,282330,138930,001460,001040,079160,000120,011150,...,004800,094280,093370,081660,005870,079980,005010,069260,000540,003280
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,21.781250,9.784632,0.206771,21.727911,7.623282,41.197994,12.800569,-1145.454545,70.243902,22.755906,...,8.872721,16.840731,32.131661,16.174757,8.974359,16.178737,29.362101,8.756254,4.056532,-1.563559
2018-01-03,21.625000,9.532039,0.206104,20.902147,7.526273,40.791435,12.909399,-1074.242424,70.731707,22.559055,...,9.415273,18.798956,32.601881,16.990291,9.014423,16.409861,28.705441,8.756254,4.033613,-1.583686
2018-01-04,22.093750,9.625100,0.200768,20.695706,7.502021,40.723675,12.764344,-1084.848485,70.731707,22.362205,...,9.064179,17.558747,31.661442,16.582524,8.814103,16.486903,28.799250,8.613295,3.964859,-1.555085
2018-01-05,22.656250,9.625100,0.194765,20.644096,7.502021,40.655915,12.655591,-1092.424242,69.024390,22.401575,...,9.128034,17.428198,31.818182,16.737864,8.733974,16.255778,29.174484,8.756254,4.094729,-1.567797
2018-01-08,22.281250,9.638394,0.194765,20.540875,7.663703,40.655915,12.583064,-1083.333333,67.804878,22.007874,...,8.745011,17.885117,31.065831,16.407767,8.453526,16.024653,28.799250,9.042173,4.094729,-1.555085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,13.546875,7.351768,0.114458,22.502064,6.200485,33.066811,9.899631,-650.000000,83.170732,18.937008,...,6.363931,14.229765,23.636364,48.932039,6.314103,11.479199,22.420263,9.274482,3.422460,-0.859110
2018-12-24,13.484375,7.218825,0.111657,21.727911,6.200485,33.202331,9.718274,-640.909091,81.463415,18.543307,...,6.439435,13.838120,22.978056,47.572816,6.474359,11.679507,22.514071,9.399571,3.453018,-0.843220
2018-12-26,13.468750,7.125764,0.108855,21.676301,6.143897,33.676650,9.536994,-612.878788,80.487805,17.834646,...,6.417862,13.577023,22.319749,47.572816,6.498397,11.587057,21.857411,9.506791,3.414820,-0.809322


### 상장폐지 데이터

In [28]:
df = fdr.DataReader('036360', exchange='krx-delisting')

In [3]:
import io

In [12]:
url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&market_gubun=STK&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&isu_srt_cd=&fromdate=20190528&todate=20200528&del_cd=1&pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'
url

'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&market_gubun=STK&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&isu_srt_cd=&fromdate=20190528&todate=20200528&del_cd=1&pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

In [40]:
url="http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=csv&url=MKD/04/0406/04060600/mkd04060600&market_gubun=STK&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&isu_srt_cd=&fromdate=20190528&todate=20200528&del_cd=1&pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp"

In [65]:
url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
            'name=fileDown&filetype=xls&url=MKD/04/0406/04060600/mkd04060600&' \
            'market_gubun=STK&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&' \
            'isu_srt_cd=&fromdate=19900101&todate=22001231&del_cd=1&' \
            'pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060600%2FMKD04060600.jsp'

In [66]:

header_data = {
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}
r = requests.get(url, headers=header_data)

In [67]:
r.text

'NVko1tOiUiiHilrxo5KmurLe/PpqGWWpv+KOM1N9+vE1kpexJeSsEGVOPbvZxNGKEJOHny+8gy4fMhDLnUBNLdjS4N1nyXBU5uWQkuvx1pyHoFlCEPBLrVysTCF5FoZYS7sGViChsERpcYp4X9Rdb3sVqqwvPFRGOUAaw2puGWkn9fzfNsrTMFtYIkF3sQdvNI0xODU9oUfInTojpThLSfEKHNnnBAxtuHqaeNwCT+5qc+c9hJkxPdevhqYaybbstQceipR5gDB9bN+o8gNz31zy7WQvgDcXUnUC76gct2nxpOzk66vKnAfz3vgreZnthUGhXR40q5RK6oCkiMCUQdJzsYz7c2K4VfFRCNseQ++AbOf0FchYHKMOHk+y+K8D0qorCN7b93ZSYuURi2h4Okkm5VRcUtD2fUnp4P6Zqbper/23Z+6ATGmCHGq9su0eE5sjVTJvyek2ND4WdqkG0w=='

In [68]:

form_data = {'code': r.text}
header_data = {
    'Referer': 'http://marketdata.krx.co.kr/contents/MKD/04/0406/04060600/MKD04060600.jsp',
    'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',
}

In [69]:
url = 'http://file.krx.co.kr/download.jspx'
r_ = requests.post(url, data=form_data, headers=header_data)

In [70]:
pd.read_excel(io.BytesIO(r_.content))

,종목코드,기업명,폐지일,폐지사유
0,A083380,동북아13호선박투자,2020/05/04,존속기간 만료
1,A159650,하이골드8호,2020/04/27,감사의견 부적정
2,A027390,한화갤러리아타임월드,2020/04/16,지주회사(최대주주등)의 완전자회사화 등
3,A011160,두산건설,2020/03/24,지주회사(최대주주등)의 완전자회사화 등
4,A079440,오렌지라이프,2020/02/14,지주회사(최대주주등)의 완전자회사화 등
...,...,...,...,...
527,A034370,럭키소재,1991/11/11,해산 사유 발생
528,A028460,태평양건설,1991/10/05,영업활동정지 6월 계속
529,A028450,금성투자금융,1991/09/02,해산 사유 발생
530,A028440,삼화,1991/07/12,감사의견 의견거절


In [83]:
kospiDf = fdr.StockListing('KOSPI')

codes = kospiDf.dropna()['Symbol'].tolist()

In [94]:
kospiDf[kospiDf.apply(lambda x:'호' in x['Name'],axis=1)].sample(10)

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
571,083370,KOSPI,동북아12호,운송장비 임대업,운수장비 임대,2005-08-29,12월,고성준,NaN,부산광역시
2104,71205B98,KOSPI,키움히어로즈유럽오피스부동산3호 A,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2286,74202767,KOSPI,하나대체티마크그랜드부동산1호 F,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1744,74701786,KOSPI,이지스196호 Class A,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1750,74701B9A,KOSPI,이지스299호A,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2294,172580,KOSPI,하이골드12호,운송장비 임대업,NaN,2013-07-12,12월,오운용,NaN,제주특별자치도
2503,008770,KOSPI,호텔신라,기타 상품 전문 소매업,"면세판매,관광숙박,외식사업,예식업",1991-03-12,12월,이부진,http://www.hotelshilla.net,서울특별시
1619,75602782,KOSPI,유경공모부동산2호ClassA,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2105,71206B98,KOSPI,키움히어로즈유럽오피스부동산3호 C-I,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2101,71202B98,KOSPI,키움히어로즈유럽오피스부동산1호 C-I,NaN,NaN,NaT,NaN,NaN,NaN,NaN


""


In [ ]:
#아이투자

# url = 'http://www.itooza.com/vclub/y10_page.php?cmp_cd=005930&mode=dy&ss=10&sv=2&lsmode=1&lkmode=1&pmode=1&exmode=1&accmode=1'

# source = requests.get(url).content

# soup= bs(source,'lxml')

# soup

# soup.findAll("div",{"class":"body"})